# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [2]:

# Load the CSV file created in part 1 into a DataFrame.
city_data_df = pd.read_csv("C:/Users/hbour/OneDrive/Desktop/class work/python_api_challenge/output_data/cities.csv")
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
city_plot=city_data_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    size="Humidity",
    color="blue",
    alpha=0.5,
    colorbar=False,
    tiles="OSM",
    title="City Latitude vs. Longitude",
    grid=True,
    frame_height=500,
    frame_width=700
)

# Display the map
city_plot

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [x]   (y,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 100 degrees Fahrenheit but higher than 70; Wind speed less than 10 mph; Zero cloudiness.
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 37) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    #print(f"Requesting URL: {base_url} with parameters: {params}")

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nalut - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
narwar - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
bilma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
146,puerto del rosario,ES,28.5004,-13.8627,73,Hotel Tamasite
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    size="Humidity",
    color="blue",
    alpha=0.5,
    colorbar=False,
    tiles="OSM",
    title="Hotel Locations",
    hover_cols=["Hotel Name", "Country"],
    frame_height=500,
    frame_width=700
)


# Display the map
hotel_plot


:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [x]   (y,Humidity,Hotel Name)